<a href="https://colab.research.google.com/github/21-LAKSHMI/AI-AND-ML/blob/main/Song_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 8.0 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.whl size=78388 sha256=63032380a689a3909eb592bcf7f706e8c8c9bb2e69df9d36ec2e3239e51fb418
  Stored in directory: /root/.cache/pip/wheels/1b/0c/20/a3b3daa1f9b65c8b0445729f94740ec335d0f86f1066c5c414
  Created wheel for julius: filename=julius-0.2.7-

In [2]:
import os
import zipfile
from demucs.separate import main

# ---- CONFIG ----
zip_path = "/content/Rap song.zip"   # your zip with 20 songs
extract_dir = "/content/songs"    # where to extract songs
output_dir = "/content/output_O"            # where to save separated files

# 1. Unzip all songs
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ Extracted all songs to {extract_dir}")

# 2. Get all audio files from extracted folder
audio_files = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith(('.mp3', '.wav', '.flac', '.ogg')):  # supported formats
            audio_files.append(os.path.join(root, file))

print(f"🎶 Found {len(audio_files)} songs in dataset.")

# 3. Run Demucs on each audio file
for idx, input_audio in enumerate(audio_files, 1):
    print(f"🔄 Processing {idx}/{len(audio_files)}: {os.path.basename(input_audio)}")

    main([
        "-n", "htdemucs_ft",        # Faster model
        "--two-stems", "vocals",    # Only vocals vs music
        "--mp3",                    # Save as MP3
        "--jobs", "4",              # Parallel jobs
        "--device", "cuda",         # Use GPU (change to "cpu" if no GPU)
        "-o", output_dir,
        input_audio
    ])

print(f"\n✅ All {len(audio_files)} songs processed! Separated files saved in {output_dir}")


✅ Extracted all songs to /content/songs
🎶 Found 21 songs in dataset.
🔄 Processing 1/21: Engeyum Eppothum.mp3
Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/f7e0c4bc-ba3fe64a.th" to /root/.cache/torch/hub/checkpoints/f7e0c4bc-ba3fe64a.th


100%|██████████| 80.2M/80.2M [00:00<00:00, 98.8MB/s]


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/d12395a8-e57c48e6.th" to /root/.cache/torch/hub/checkpoints/d12395a8-e57c48e6.th


100%|██████████| 80.2M/80.2M [00:00<00:00, 178MB/s]


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/92cfc3b6-ef3bcb9c.th" to /root/.cache/torch/hub/checkpoints/92cfc3b6-ef3bcb9c.th


...

🔄 Processing 20/21: Club-La-Mabula.mp3
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/output_O/htdemucs_ft
Separating track /content/songs/Rap song/Club-La-Mabula.mp3


100%|██████████████████████████████████████████████████████████████████████| 216.45/216.45 [00:08<00:00, 25.50seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 216.45/216.45 [00:08<00:00, 25.62seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 216.45/216.45 [00:08<00:00, 25.33seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 216.45/216.45 [00:08<00:00, 25.16seconds/s]


🔄 Processing 21/21: AK - The Tiger.mp3
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/output_O/htdemucs_ft
Separating track /content/songs/Rap song/AK - The Tiger.mp3


100%|██████████████████████████████████████████████████████████████████████| 204.75/204.75 [00:08<00:00, 25.28seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 204.75/204.75 [00:07<00:00, 25.61seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 204.75/204.75 [00:08<00:00, 25.31seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 204.75/204.75 [00:08<00:00, 25.34seconds/s]



✅ All 21 songs processed! Separated files saved in /content/output_O


In [3]:
import os
import zipfile
from demucs.separate import main

# ---- CONFIG ----
zip_path = "/content/melody_songs.zip"   # your zip with 20 songs
extract_dir = "/content/melody_songs"    # where to extract songs
output_dir = "/content/melody_output_O"            # where to save separated files

# 1. Unzip all songs
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"✅ Extracted all songs to {extract_dir}")

# 2. Get all audio files from extracted folder
audio_files = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith(('.mp3', '.wav', '.flac', '.ogg')):  # supported formats
            audio_files.append(os.path.join(root, file))

print(f"🎶 Found {len(audio_files)} songs in dataset.")

# 3. Run Demucs on each audio file
for idx, input_audio in enumerate(audio_files, 1):
    print(f"🔄 Processing {idx}/{len(audio_files)}: {os.path.basename(input_audio)}")

    main([
        "-n", "htdemucs_ft",        # Faster model
        "--two-stems", "vocals",    # Only vocals vs music
        "--mp3",                    # Save as MP3
        "--jobs", "4",              # Parallel jobs
        "--device", "cuda",         # Use GPU (change to "cpu" if no GPU)
        "-o", output_dir,
        input_audio
    ])

print(f"\n✅ All {len(audio_files)} songs processed! Separated files saved in {output_dir}")


✅ Extracted all songs to /content/melody_songs
🎶 Found 20 songs in dataset.
🔄 Processing 1/20: Guru (Tamil) - Aaruyirae Video  A.R. Rahman.mp3
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/melody_output_O/htdemucs_ft
Separating track /content/melody_songs/melody songs/Guru (Tamil) - Aaruyirae Video  A.R. Rahman.mp3


100%|██████████████████████████████████████████████████████████████████████| 286.65/286.65 [00:12<00:00, 23.71seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 286.65/286.65 [00:11<00:00, 25.53seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 286.65/286.65 [00:11<00:00, 25.27seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 286.65/286.65 [00:11<00:00, 25.45seconds/s]


🔄 Processing 2/20: Varaha Nadhikarai (From Sangamam).mp3
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/melody_output_O/htdemucs_ft
Separating track /content/melody_songs/melody songs/Varaha Nadhikarai (From Sangamam).mp3


100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:14<00:00, 25.61seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:14<00:00, 25.45seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:14<00:00, 25.41seconds/s]
100%|██████████████████████████████████████████████████████████████████████| 380.25/380.25 [00:14<00:00, 25.71seconds/s]


🔄 Processing 3/20: Maragatha Naanayam  Nee Kavithaigala Song with Lyrics  Aadhi, Nikki Galrani  Dhibu Ninan Thomas.mp3
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/melody_output_O/htdemucs_ft
Separating track /content/melody_songs/melody songs/Maragatha Naanayam  Nee Kavithaigala Song with Lyrics  Aadhi, Nikki Galrani  Dhibu Ninan Thomas.mp3


...

🔄 Processing 19/20: Pannaiyaarum Padminiyum - Onakkaaga Poranthaenae Video  Vijay Sethupathi.mp3
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/melody_output_O/htdemucs_ft
Separating track /content/melody_songs/melody songs/Pannaiyaarum Padminiyum - Onakkaaga Poranthaenae Video  Vijay Sethupathi.mp3


100%|██████████████████████████████████████████████| 198.89999999999998/198.89999999999998 [00:07<00:00, 25.63seconds/s]
100%|██████████████████████████████████████████████| 198.89999999999998/198.89999999999998 [00:07<00:00, 25.67seconds/s]
100%|██████████████████████████████████████████████| 198.89999999999998/198.89999999999998 [00:07<00:00, 25.49seconds/s]
100%|██████████████████████████████████████████████| 198.89999999999998/198.89999999999998 [00:07<00:00, 25.53seconds/s]


🔄 Processing 20/20: Irudhi Suttru Video Songs  Ey Sandakaara Full Video SongR Madhavan,Ritika SinghSanthosh Narayanan.mp3
Selected model is a bag of 4 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/melody_output_O/htdemucs_ft
Separating track /content/melody_songs/melody songs/Irudhi Suttru Video Songs  Ey Sandakaara Full Video SongR Madhavan,Ritika SinghSanthosh Narayanan.mp3


100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 25.51seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 25.60seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 25.44seconds/s]
100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:09<00:00, 25.57seconds/s]



✅ All 20 songs processed! Separated files saved in /content/melody_output_O


In [4]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler

# Define paths to the output directories
rap_output_dir = "/content/output_O/htdemucs_ft"
melody_output_dir = "/content/melody_output_O/htdemucs_ft"

# Create empty lists to store features and labels
features = []
labels = []

In [5]:
# Process rap songs
for song_folder in os.listdir(rap_output_dir):
    song_path = os.path.join(rap_output_dir, song_folder, "vocals.mp3")
    if os.path.exists(song_path):
        try:
            y, sr = librosa.load(song_path, sr=None)
            # Extract features (MFCCs, Chroma, Spectral Centroid)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

            # Aggregate features (e.g., mean)
            features.append(np.mean(mfccs, axis=1))
            features.append(np.mean(chroma, axis=1))
            features.append(np.mean(spec_centroid, axis=1))
            labels.append('rap')
        except Exception as e:
            print(f"Error processing {song_path}: {e}")


# Process melody songs
for song_folder in os.listdir(melody_output_dir):
    song_path = os.path.join(melody_output_dir, song_folder, "vocals.mp3")
    if os.path.exists(song_path):
        try:
            y, sr = librosa.load(song_path, sr=sr)
            # Extract features (MFCCs, Chroma, Spectral Centroid)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

            # Aggregate features (e.g., mean)
            features.append(np.mean(mfccs, axis=1))
            features.append(np.mean(chroma, axis=1))
            features.append(np.mean(spec_centroid, axis=1))
            labels.append('melody')
        except Exception as e:
            print(f"Error processing {song_path}: {e}")


In [7]:
# Create empty lists to store features and labels
features = []
labels = []

# Process rap songs
for song_folder in os.listdir(rap_output_dir):
    song_path = os.path.join(rap_output_dir, song_folder, "vocals.mp3")
    if os.path.exists(song_path):
        try:
            y, sr = librosa.load(song_path, sr=None)
            # Extract features (MFCCs, Chroma, Spectral Centroid)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

            # Aggregate and combine features
            combined_features = np.concatenate((np.mean(mfccs, axis=1),
                                                np.mean(chroma, axis=1),
                                                np.mean(spec_centroid, axis=1)))
            features.append(combined_features)
            labels.append('rap')
        except Exception as e:
            print(f"Error processing {song_path}: {e}")


# Process melody songs
for song_folder in os.listdir(melody_output_dir):
    song_path = os.path.join(melody_output_dir, song_folder, "vocals.mp3")
    if os.path.exists(song_path):
        try:
            y, sr = librosa.load(song_path, sr=sr)
            # Extract features (MFCCs, Chroma, Spectral Centroid)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

            # Aggregate and combine features
            combined_features = np.concatenate((np.mean(mfccs, axis=1),
                                                np.mean(chroma, axis=1),
                                                np.mean(spec_centroid, axis=1)))
            features.append(combined_features)
            labels.append('melody')
        except Exception as e:
            print(f"Error processing {song_path}: {e}")

# Convert lists to NumPy arrays
X = np.array(features)
y = np.array(labels)

# Preprocess features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Features shape:", X_scaled.shape)
print("Labels shape:", y.shape)

Features shape: (41, 26)
Labels shape: (41,)


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical

# Convert labels to numerical categorical labels
y_encoded = to_categorical(y == 'melody', num_classes=2)

# Reshape features for CNN input (add a channel dimension)
X_scaled_cnn = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_scaled_cnn.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_scaled_cnn, y_encoded, epochs=50, batch_size=32)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 847ms/step - accuracy: 0.4223 - loss: 0.7449
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7274 - loss: 0.6287
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8770 - loss: 0.5630
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9304 - loss: 0.5021
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8933 - loss: 0.4612
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9037 - loss: 0.4141
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9304 - loss: 0.3678
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9571 - loss: 0.3277
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9571 - loss: 0.3076
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9733 - loss: 0.2872
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9733 - loss: 0.2712
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9571 - loss: 0.2384


In [11]:
# Define the path to the new song file
new_song_path = "/content/melody_songs/melody songs/Maamannan - Nenjame Nenjame Video  Udhayanidhi Stalin  Vadivelu  A.R Rahman.mp3"

# Load the new song
try:
    y_new, sr_new = librosa.load(new_song_path, sr=sr) # Use the same sample rate as training data
except Exception as e:
    print(f"Error loading new song: {e}")
    y_new, sr_new = None, None # Set to None if loading fails

if y_new is not None:
    # Extract features (MFCCs, Chroma, Spectral Centroid)
    try:
        mfccs_new = librosa.feature.mfcc(y=y_new, sr=sr_new, n_mfcc=13)
        chroma_new = librosa.feature.chroma_stft(y=y_new, sr=sr_new)
        spec_centroid_new = librosa.feature.spectral_centroid(y=y_new, sr=sr_new)

        # Aggregate and combine features
        combined_features_new = np.concatenate((np.mean(mfccs_new, axis=1),
                                            np.mean(chroma_new, axis=1),
                                            np.mean(spec_centroid_new, axis=1)))

        # Reshape for prediction (1 sample, n_features)
        X_new = combined_features_new.reshape(1, -1)

        # Scale the features using the fitted scaler
        X_new_scaled = scaler.transform(X_new)

        print("New song features shape:", X_new_scaled.shape)

    except Exception as e:
        print(f"Error extracting features from new song: {e}")
        X_new_scaled = None # Set to None if feature extraction fails
else:
    X_new_scaled = None # Set to None if loading failed

New song features shape: (1, 26)


In [12]:
# Make a prediction
if X_new_scaled is not None:
    predictions = model.predict(X_new_scaled)
    predicted_class = np.argmax(predictions, axis=1)[0]

    # Map the predicted class back to the genre label
    genre_mapping = {0: 'rap', 1: 'melody'}
    predicted_genre = genre_mapping[predicted_class]

    print(f"The predicted genre for the new song is: {predicted_genre}")
else:
    print("Could not predict the genre of the new song due to previous errors.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
The predicted genre for the new song is: melody
